# Ques 1

In [121]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize.casual import EMOTICON_RE
from nltk.tokenize import RegexpTokenizer as reg
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

path = '/content/drive/MyDrive/Dataset/Humor,Hist,Media,Food'
file_names = []
all_txt = []
dataset = []

for f in os.listdir(path):
    all_txt.append(os.path.join(path, f))
    file_names.append(f)

for i in all_txt:
    dataset.append(open(i,'r',encoding="iso-8859-1").read())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [122]:
print(len(dataset))

1133


In [123]:
def preprocess(data) : 
    preprocessed = []
    cache = set(stopwords.words("english"))
    porter = PorterStemmer()

    for file in data:
        preproc = re.sub('[^a-zA-Z0-9]',' ',file)
        preproc = preproc.lower()
        preproc = preproc.split()
        preproc = [porter.stem(word) for word in preproc if not word in cache]
        preprocessed.append(preproc)
        
    return preprocessed

In [125]:
def unigram_index(data) : 
    sol={}
    all_docs = []
    doc_idx = 0
    for text in (data) : 
        # print(text)
        for word in text :
            # print(word) 
            if word in sol : 
                if(doc_idx in sol[word]) : 
                    continue
                else : 
                    sol[word].append(doc_idx)
            else : 
                sol[word] = []
                sol[word].append(doc_idx)
        all_docs.append(doc_idx)
        doc_idx += 1
    return sol, all_docs

In [126]:
preprocessed_data = preprocess(dataset)
dict_unigram = {}
dict_unigram, all_docs = unigram_index(preprocessed_data)

In [127]:
def and_op(doc1, doc2) : 
    doc1.sort()
    doc2.sort()

    ans = []
    tot = 0
    i=0
    j=0

    while(i<len(doc1) and j<len(doc2)) :
        if(doc1[i] == doc2[j]) :
            ans.append(doc1[i]) 
            i+=1
            j+=1
        elif(doc1[i] < doc2[j]) :
            i+=1
        else :
            j += 1
        tot += 1
    
    return ans, tot

In [128]:
def or_op(doc1, doc2):
    doc1.sort()
    doc2.sort()
    i=0
    j=0
    ans = []
    tot = 0
    while(i<len(doc1) and j<len(doc2)):
        if(doc1[i] == doc2[j]):
            ans.append(doc1[i])
            i+=1
            j+=1
        elif(doc1[i] < doc2[j]):
            ans.append(doc1[i])
            i+=1
        else:
            ans.append(doc2[j])
            j+=1
        tot+=1

    if(i == len(doc1)):
        while(j<len(doc2)):
            ans.append(doc2[j])
            j+=1
        
    if(j == len(doc2)) : 
        while(i<len(doc1)):
            ans.append(doc1[i])
            i+=1
    
    return ans, tot
        

In [156]:
def not_op(doc1, doc2) : 
    return list(set(doc1)-set(doc2))

In [153]:
def preprocess_query(q) : 
    porter = PorterStemmer()
    preproc = re.sub('[^a-zA-Z0-9]',' ',q)
    preproc = preproc.lower()
    preproc = preproc.split()
    preproc = [porter.stem(word) for word in preproc]

    return preproc
n = 1 

queries = []
for i in range(n) : 
    query_inp = input()
    # query_inp = "telephone,paved, roads"
    query_inp = preprocess_query(query_inp)
    input_seq = []
    for j in range(len(query_inp)-1): 
        inp = input()
        input_seq.append(inp)
    query = []
    for j in range(0, len(query_inp)) :
        query.append(query_inp[j])
        # print(j)
        # print(len(input_seq))
        if(j < len(input_seq)) : 
            x = input_seq[j].lower()
            x = x.split()
            query.extend(x)
    queries.append(query)
    # print(query)


essay thesis readers
AND
OR NOT


In [157]:
for query in queries :
    i=1

    if(query[0] not in dict_unigram) : 
        dict_unigram[query[0]] = {}
    prev = dict_unigram.get(query[0])
    tot = 0
    while(i < len(query)-1) : 
        if(query[i] == 'and') : 
            if(i+1 < len(query)-1 and query[i+1] == 'not') : 
                if(not(query[i+2] in dict_unigram)):
                    dict_unigram[query[i+2]]={}
                a2 = not_op(all_docs, dict.unigram(query[i+2]))
                v1, v2 = and_op(prev, a2)
                tot += v2
                prev = v1
                i+=3
            else : 
                a2 = dict_unigram.get(query[i+1])
                v1, v2 = and_op(prev, a2)
                tot += v2
                prev = v1
                i+=2
        
        elif(query[i] == 'or') : 
            if(i+1 < len(query)-1 and query[i+1] == 'not') : 
                if(not(query[i+2] in dict_unigram)):
                    dict_unigram[query[i+2]]={}
                a2 = not_op(all_docs, dict_unigram.get(query[i+2]))
                # a2 = list(set(all_docs)-set(dict_unigram.get(query[i+2])))
                v1, v2 = or_op(prev, a2)
                # print("v1 ", v1) 
                tot += v2
                prev = v1
                i+=3
            else : 
                # query[i+1] = preproc_word(query[i+1])
                a2 = dict_unigram.get(query[i+1])
                # print(prev)
                # print(a2)
                v1, v2 = or_op(prev, a2)
                print("v1 ", v1)
                tot += v2
                prev = v1
                i+=2

    print("Total Number of Documents ", len(prev))
    print("Total number of comparisons ", tot)



Total Number of Documents  1051
Total number of comparisons  41
